In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
questions = pd.read_csv("input/TrainQuestions.csv", encoding='latin1')
answers = pd.read_csv("input/TrainAnswers.csv", encoding='latin1')
tags = pd.read_csv("input/TrainTags.csv", encoding='latin1')
users = pd.read_csv("users.csv", encoding='latin1')


In [9]:
def feature(data):
    global users
    feat = [1]
    #Getting Body related Features
    soup = BeautifulSoup(data['Body'],"lxml")
    numtags = len(set([tag.name for tag in soup.body.findAll(True)]))
    isimage = 1 if soup.body.findAll("img") else 0
    islink = 1 if soup.body.findAll("a") else 0
    iscode = 1 if (soup.body.findAll("code") or soup.body.findAll("pre")) else 0
    feat.extend([numtags,isimage,islink,iscode])
    
    #Getting number of words in title
    titleWords = list(filter(None, re.split("[, \-!?:]+", data['Title'])))
    feat.append(len(titleWords))

    #Getting User related Features
    badges = -1
    reputation = -1
    accept_rate = -1
    if not np.isnan(data['OwnerUserId']) and int(data['OwnerUserId']) in users['User Id'].values:
        ownerdata = users.loc[users['User Id']==int(data['OwnerUserId'])].iloc[0]
        badges = ownerdata['Bronze Badges'] + ownerdata['Silver Badges'] + ownerdata['Gold Badges']
        reputation = ownerdata['Reputation']
        accept_rate = ownerdata['Accept Rate']
    feat.extend([badges, reputation, accept_rate])
    return feat

In [12]:
#X = [feature(questions.iloc[d]) for d in range(len(questions))]
X = feature(questions.iloc[0])
print(X)
print(questions.iloc[0]['OwnerUserId'])
print(users.loc[users['User Id']==1036606])
questions.iloc[0]

[1, 2, 0, 0, 1, 36, 520, 83.0]
1036606.0
   Unnamed: 0  Bronze Badges  Silver Badges  Gold Badges  Account Id  \
9           9             25             11            0     1010392   

  Is Employee  Reputation  User Id  Accept Rate  
9       False         520  1036606         83.0  


Unnamed: 0                                                 234318
Id                                                        8767810
OwnerUserId                                           1.03661e+06
CreationDate                                 2012-01-07T06:14:39Z
ClosedDate                                                    NaN
Score                                                           1
Title                Compiling a static QT application on Windows
Body            <p>I am new to QT and I am enjoying the experi...
Name: 0, dtype: object

In [6]:
userids = questions['OwnerUserId'].unique()
owneruserids = [int(x) for x in userids if not np.isnan(x)]
scrapeduserids = users['User Id'].unique()

print(len(userids),len(users))
not_present = 0
for i in owneruserids:
    if i not in users['User Id'].values:
        if not_present == 0:
            print (i)
        not_present += 1
print(not_present)
print(users.loc[users['User Id']==557846])


144938 144716
557846
221
Empty DataFrame
Columns: [Unnamed: 0, Bronze Badges, Silver Badges, Gold Badges, Account Id, Is Employee, Reputation, User Id, Accept Rate]
Index: []
